Or just flesh out ssa is functional programming
mimir

https://c9x.me/compile/doc/il.html
https://pypy.org/posts/2024/07/toy-abstract-interpretation.html
https://gist.github.com/tekknolagi/4425b28d5267e7bae8b0d7ef8fb4a671

Yea man, I need a compiler project.

https://dl.acm.org/doi/abs/10.1145/3689717 A Low-Level Look at A-Normal Form

Convert qbe to block args form


A lowkey branching.
No yea, that ought to be possible.
If we have direct jmps to constants.



- Explicitize memory
- phi to block args
- CSE

- memset tearing my hair out


- nice lbock rpinting
- QBE laternative
- take in correspondance block by block

+ alternate toolchains
  - cbmcpy - output pcode. asmc. goto. Could be nice to actually link into kani
  - leancall + pypcode + mriscx

Kaiju papers and venues


DEMO BY NEXT TUESDAY
3 assembly instructions.
LETS GOOOOOO


So, maybe the CHC form is more flexible.
a relational execution spec
trans(state1, state2) :- not_correspond(state1,state2)
trans(state1, state2) :- trans1(state1, state10), trans2(state2,state20), trans(state10, state20)

tr


myfun() :- loop(1,1,0, store(0,1,1,1,2,0, mem)) 
loop(i,j,k, mem) :- mem[0] != i ; mem[1] != j; mem[2] != k 
loop(i,j,k, mem) :- mem[0] = i, mem[1] = j, mem[2] = k, if k < 100; if_head(i,j,k, mem); done(i,j,k,mem) 


What about an equational product program thing
What is the FLP version of CHC modelling?

loop2(i,j,k, mem) = loop0(i,j,k)
loop2(i,j,k, mem) = loop1(mem)
loop2(i,j,k, mem) = if k < 0 then 

I don't know that I could take the 

loop(i,j,k,mem) :- loop0(i,j,k), loop1(mem), .

state is both function symbol (label ~ pc) and the arugments.
Do I need to do some functional programming  abstract machine thing? CEK etc etc. Tail calls means I don't need much of a continuation. 
call/cc is kind of just reading from pc. That's fun. Well... maybe you should capture the entire universe too?


prolog as a model checker. spin as a solver.

trans(loop, [i,j,k]) :- 


state = (lambda x, loop x, i, j, k)

loop -> if_head  ----> exists Q,

https://homes.cs.washington.edu/~djg/msr_russia2012/sangiorgi.pdf




In [9]:
%%file /tmp/foo.py

def foo(x : int):
    if x > 0:
        return x
    else:
        assert False, "x must be positive"

Overwriting /tmp/foo.py


In [10]:
! esbmc /tmp/foo.py --function foo

ESBMC version 8.0.0 64-bit x86_64 linux
Target: 64-bit little-endian x86_64-unknown-linux with esbmclibc
Parsing /tmp/foo.py



Type checking warning:

<unknown>:45: SyntaxWarning: invalid escape sequence '\d'
Converting
Generating GOTO Program
GOTO program creation time: 0.406s
GOTO program processing time: 0.004s
Starting Bounded Model Checking
Symex completed in: 0.000s (47 assignments)
Caching time: 0.000s (removed 0 assertions)
Slicing time: 0.000s (removed 43 assignments)
Generated 2 VCC(s), 2 remaining after simplification (4 assignments)
No solver specified; defaulting to z3
Encoding remaining VCC(s) using bit-vector/floating-point arithmetic
Encoding to solver time: 0.000s
Solving with solver Z3 v4.8.12
Runtime decision procedure: 0.006s
Building error trace

[Counterexample]


State 1 file /tmp/foo.py line 6 column 8 function foo thread 0
----------------------------------------------------
Violated property:
  file /tmp/foo.py line 6 column 8 function foo
  x must be positive
  x > 0


VERIFICATION FAILED


Build all cuts to cuts
Correspond those. Instead of block by block
Join / merge of those pathways.
I wanted this to maybe write specs /ghost code in a assembly anyway



In [1]:
%%file /tmp/myfun.s

.global myfun
.equ i, %rdi
.equ j, %rsi
.equ k, %rdx

loop:
    ret # todo
myfun:
    mov 1, i
    mov 1, j
    mov 0, k
    jmp loop
    nop

Writing /tmp/myfun.s


In [2]:
! gcc -c -o /tmp/myfun /tmp/myfun.s

In [3]:
import kdrag.contrib.pcode as pcode
from kdrag.all import *
ctx = pcode.BinaryContext("/tmp/myfun")

memstate0 = pcode.MemState.Const("memstate0")
memstate1 = ctx.execute_block(memstate0, ctx.loader.find_symbol("myfun").rebased_addr)
#kd.tuple_(0xdeadbeef, memstate1)

Unexpected SP conflict


highexec(high(mem)) == high(lowexec(mem))
So how should I take these in?


Next step up, a two block thing.
A simple if branching

two variables of state

Why was I thrashing so hard on getting data out. My api isn't great.
Make ctx a part of memstate.




# 1 Block 1 Var

In [46]:
%%file /tmp/simple.s
.global simple
done:
    nop
simple:
    mov $1, %rdi
    add %rdi, %rax
    jmp done

Overwriting /tmp/simple.s


In [47]:
! gcc -c -o /tmp/simple /tmp/simple.s

In [48]:
from kdrag.contrib.pcode import BinaryContext
ctx = BinaryContext("/tmp/simple")
memstate0 = pcode.MemState.Const("memstate0")
memstate1 = ctx.execute_block(memstate0, ctx.loader.find_symbol("simple").rebased_addr)


#step_low = memstate1[0].memstate.register.to_expr()
memstate1

Unexpected SP conflict


[SimState(memstate=MemState((let ((a!1 (bvadd ((_ zero_extend 1) (select64le (register memstate0) &RAX))
                   ((_ zero_extend 1) #x0000000000000001)))
       (a!4 (and (bvslt #x0000000000000000
                        (select64le (register memstate0) &RAX))
                 (bvslt #x0000000000000000 #x0000000000000001)))
       (a!5 (bvslt #x0000000000000000
                   (bvadd (select64le (register memstate0) &RAX)
                          #x0000000000000001)))
       (a!8 (bvsgt #x0000000000000000
                   (bvadd (select64le (register memstate0) &RAX)
                          #x0000000000000001)))
       (a!9 (= #x0000000000000000
               (bvadd (select64le (register memstate0) &RAX) #x0000000000000001)))
       (a!10 (bvand (bvadd (select64le (register memstate0) &RAX)
                           #x0000000000000001)
                    #x00000000000000ff)))
 (let ((a!2 (ite (not (= ((_ extract 64 64) a!1) #b0)) #x01 #x00))
       (a!11 (= #x00 (

In [49]:
high_init = ctx.get_reg(memstate0, "RAX")
high_lowstep = ctx.get_reg(memstate1[0].memstate, "RAX")
kd.prove(high_init + 1 == high_lowstep)

|= select64le(register(memstate0), &RAX) + 1 ==
1 + select64le(register(memstate0), &RAX)

In [34]:
high_init

select64le(register(memstate0), &RAX)

In [35]:
high_lowstep

select64le(register(memstate0), &RAX) +
select64le(ram(memstate0), 1)

# 2 block 1 var
TODO:
    - backjump is not sufficient. Make take it start and end addr


In [13]:
%%file /tmp/simple.s
.equ i, %rdi
.equ j, %rax

.global simple
.global block2
done:
    ret
block2:
    add i, j
    jmp done
simple:
    mov $1, i
    jmp block2


Overwriting /tmp/simple.s


In [14]:
! gcc -c -o /tmp/simple /tmp/simple.s

In [22]:
from kdrag.contrib.pcode import BinaryContext
import kdrag.contrib.pcode as pcode
from kdrag.all import *
ctx = BinaryContext("/tmp/simple")
memstate0 = pcode.MemState.Const("memstate0")
memstate1 = ctx.execute_block(memstate0, ctx.loader.find_symbol("simple").rebased_addr)
memstate2 = ctx.execute_block(memstate0, ctx.loader.find_symbol("block2").rebased_addr)
j0 = ctx.get_reg(memstate0, "RAX")
i0 = ctx.get_reg(memstate0, "RDI")
i1 = ctx.get_reg(memstate1[0].memstate, "RDI")
j1 = ctx.get_reg(memstate1[0].memstate, "RAX")

def trans1(i0,j0,i1,j1):
    return smt.And(i1 == 1, j1 == j0)
# simple -> block2
kd.prove(trans1(i0,j0,i1,j1))
#kd.prove(i1 == 1)
#kd.prove(smt.And(j0 == j1, i1 == 1))

def trans2(i0,j0,i2,j2):
    return smt.And(i2 == i0, j2 == j0 + i0)
# block2 -> done
i2 = ctx.get_reg(memstate2[0].memstate, "RDI")
j2 = ctx.get_reg(memstate2[0].memstate, "RAX")
kd.prove(trans2(i0,j0,i2,j2))
#kd.prove(smt.And(j2 == j0 + i0, i2 == i0))


Unexpected SP conflict


|= And(select64le(register(memstate0), &RDI) ==
    select64le(register(memstate0), &RDI),
    select64le(register(memstate0), &RAX) +
    select64le(register(memstate0), &RDI) ==
    select64le(register(memstate0), &RAX) +
    select64le(register(memstate0), &RDI))

# If Chain


In [25]:
%%file /tmp/simple.s
.equ i, %rdi
.equ j, %rax

.global simple
.global block2
done:
    ret
then:
    add i, j
    jmp done
simple:
    mov $1, i
    cmp $0, j
    je then
else:
    sub i, j
    jmp done

Overwriting /tmp/simple.s


In [26]:
! gcc -c -o /tmp/simple /tmp/simple.s

In [ ]:
import kdrag.contrib.pcode as pcode
from kdrag.all import *
ctx = pcode.BinaryContext("/tmp/simple")
memstate0 = pcode.MemState.Const("memstate0")
memstate1 = ctx.execute_block(memstate0, ctx.loader.find_symbol("simple").rebased_addr)
memstate2 = ctx.execute_block(memstate0, ctx.loader.find_symbol("then").rebased_addr)
memstate3 = ctx.execute_block(memstate0, ctx.loader.find_symbol("else").rebased_addr)

i = ctx.get_reg(memstate0, "RDI")
j = ctx.get_reg(memstate0, "RAX")
def simple_then(i0,j0,i1,j1):
    return smt.And(i1 == 1, j0 == 0, j1 == j0)
# low -> high. shallowly described
i1 = ctx.get_reg(memstate1[0].memstate, "RDI")
j1 = ctx.get_reg(memstate1[0].memstate, "RAX")
kd.prove(smt.Implies(smt.And(memstate1[0].path_cond), simple_then(i,j,i1,j1)))
#kd.prove(simple_then(i,j,i1,j1))
#smt.simplify(smt.And(memstate1[0].path_cond))
#simple_then(i,j,i1,j1)
def simple_else(i0,j0,i2,j2):
    return smt.And(i1 == 1, j0 != 0, j1 == j0)
i2 = ctx.get_reg(memstate1[1].memstate, "RDI")
j2 = ctx.get_reg(memstate1[1].memstate, "RAX")
kd.prove(smt.Implies(smt.And(memstate1[1].path_cond), simple_else(i,j,i2,j2)))

def else_done(i0,j0,i3,j3):
    return smt.And(i3 == i0, j3 == j0 - i0)
i3 = ctx.get_reg(memstate3[0].memstate, "RDI")
j3 = ctx.get_reg(memstate3[0].memstate, "RAX")
kd.prove(smt.Implies(smt.And(memstate3[0].path_cond), else_done(i,j,i3,j3)))

def then_done(i0,j0,i4,j4):
    return smt.And(i4 == i0, j4 == j0 + i0)
i4 = ctx.get_reg(memstate2[0].memstate, "RDI")
j4 = ctx.get_reg(memstate2[0].memstate, "RAX")
kd.prove(smt.Implies(smt.And(memstate2[0].path_cond), then_done(i,j,i4,j4)))



Unexpected SP conflict


|= Implies(And,
        And(select64le(register(memstate0), &RDI) ==
            select64le(register(memstate0), &RDI),
            select64le(register(memstate0), &RAX) +
            select64le(register(memstate0), &RDI) ==
            select64le(register(memstate0), &RAX) +
            select64le(register(memstate0), &RDI)))

In [ ]:
def execute_break()

def transitions(ctx, labels : list[str | int]):
    res = []
    for label in labels:
        memstate = ctx.execute_block(memstate0, ctx.loader.find_symbol(label).rebased_addr)[0]
        res.append((label, memstate))
    return res


# Bits
How to take in data:
- fully encoded into z3
- Somewhat meta
- i,j,k tuples vs Block tagged ADT vs individual variable step functions
- New conrete syntax or no.


In [ ]:
def bisim(high, low_step, high_step):
    mem = Memstate.Const("mem")
    kd.prove(smt.ForAll([mem], ))

In [ ]:
{
    "loop" : lambda ijk: (ijk[0], ijk[1], ijk[2]),
    "if_head" : lambda ijk: (ijk[0], ijk[1], ijk[2]),
}

In [ ]:
def pymodel(label, ijk):
    if label == "loop":
        i, j, k = ijk
        return (i, j, k)
    elif label == "if_head":
        i, j, k = ijk
        return (i, j, k)
    else:
        ...


In [ ]:
def pymodel(label, mem):
    if label == "loop":
        i,j,k = mem.reg("rax"), mem.reg("rsi"), mem.reg("rdx")
        return {
            "loop" : 

        }
    elif label == "if_head":
        i,j,k = mem.reg("rax"), mem.reg("rsi"), mem.reg("rdx")
        return {
            "if_head" : (i, j, k)
        }

In [ ]:
i = lambda mem: mem.reg("rax")
pymodel = {
    "loop" : lambda mem: i(mem)
}

In [ ]:
def low_step(memstate, block_addrs):
    for addr in block_addrs:
        memstate = ctx.execute_block(memstate, addr)
    return memstate

def high_step(highstate):
    memstate = pcode.MemState.Const("memstate0")
    for block_addrs in highstate:
        memstate = low_step(memstate, block_addrs)
    return kd.tuple_(memstate.reg["rax"], memstate.reg["rbx"], memstate.reg["rcx"], memstate.reg["rdx"])

def high(memstate):
    return kd.tuple_(memstate.reg["rax"], memstate.reg["rbx"], memstate.reg["rcx"], memstate.reg["rdx"])



In [ ]:
BV64 = smt.BitVecSort(64)
MyFun = kd.Inductive("MyFun")
MyFun.declare("MyFun")
MyFun.declare("Loop", BV64, BV64, BV64, MyFun)
memstate0.


Jesus there is so much boiler plate. Generate it?
next_i(addr, i,j,k)
next_j(addr, i,j,k)
next_k(addr, i, j, k)
next_addr()


i = memstate.reg("rdi")
j = memstate.reg("rsi")
k = memstate.reg("rax")

def i(memstate):
    return memstate.reg("rdi")
def step(memstate):
    kd.symexecute(memstate)

If we have a functional relationship highstate(lowstate) (or vice versa) we can remove one foral
forall high low, relate(high, low) -> P
forall high low, high = highlow(low) -> P
forall low, P[highlow(low) / high]

P = relate()

I guess in the LTS sense, all transitions have the same label. The states are full states.
Then 
forall P1 R Q1, P2 ~> P1 -> exists Q2, Q1 ~> Q2 /\ P2 R Q2
P2 = stepP(P1)
Q2 = stepQ(Q2)

forall P1 R Q2, P2 = stepP(P1), stepP(P1) R stepQ(Q1)
Yes. That is a simulation.
And actually it's symmettric?

So
1. single "action" (maybe interactivity changes this?)
2. determinsim, Transition relation is functional
3. deriving high from low. R is functional




ijk2 = step_high(i(memstate), j(memstate), k(memstate)), 
smt.And(ijk2[0] == i(memstate1), ijk2[1] == j(memstate1), ...)



In [18]:
memstate1[0].memstate.to_expr()

TypeError: FuncDeclRef.__call__() got an unexpected keyword argument 'ram'

In [15]:
kd.tuple_(memstate1[0].pc[0], memstate1[0].memstate)

Z3Exception: Python bool, int, long or float expected

In [ ]:
def myfun():
    i = 1
    j = 1
    k = 0
    while k < 100:
        if j < 20:
            j = i
            k = k+1
        else:
            j = k
            k = k + 2
    return j

print(myfun())

In [ ]:
def myfun():
    return loop(1,1,0)

def loop(i,j,k): # orange block
    return if_head(i,j,k) if k < 100 else done(i,j,k)

def if_head(i,j,k): # red block
    return then(i,j,k) if j < 20 else else_(i,j,k)

def then(i,j,k):
    return loop(i, i, k + 1)

def else_(i,j,k):
    return loop(i, k, k + 2)

def done(i,j,k): # green block
    return j

myfun()

In [ ]:
from kdrag.all import *
Z = smt.IntSort()

# predeclare all our blocks so that we can recursively call them
myfun = smt.Function("myfun", Z)
loop = smt.Function("loop", Z,Z,Z, Z)
if_head = smt.Function("if_head", Z,Z,Z, Z)
then = smt.Function("then", Z,Z,Z, Z)
else_ = smt.Function("else_", Z,Z,Z, Z)
done = smt.Function("done", Z,Z,Z, Z)
i,j,k = smt.Ints("i j k")

myfun = kd.define("myfun", [], loop(1,1,0))
loop = kd.define("loop", [i,j,k], smt.If(k < 100, if_head(i,j,k), done(i,j,k)))
if_head = kd.define("if_head", [i,j,k], smt.If(j < 20, then(i,j,k), else_(i,j,k)))
then = kd.define("then", [i,j,k], loop(i, i, k + 1))
else_ = kd.define("else_", [i,j,k], loop(i, k, k + 2))
done = kd.define("done", [i,j,k], j)

This is so heavihanded though.
a relation is carried if
R <= T1 R T2 or something
T1 R = R T2?

https://ncatlab.org/nlab/show/bisimulation


prog = if cons(prog, prog)

```
ret(loop(I,J,K)) := None
ret(if_head(I,J,K)) := None
ret(done(I,J,K)) := Some J


next(loop(I,J,K)) := if K < 100 then if_head()
next(...) := ...
next(...) := ...
```

```
loop(i,j,k) := cons(None, if K < 100 then if_head(i,j,k) else done(i,j,k))
if_head(I,J,K) := cons(None, if j < 20 then then_block(i,j,k) else else_blk(i,j,k))
else_block(I,J,K) := cons(None, loop(i, k, k+1))
... and so on
done(i,j,k) := cons(Some j, done(i,j,k))
```


In [15]:
from kdrag.all import *
Z = smt.IntSort()
Block = kd.Inductive("Block", auto_fresh=True)
Block.declare("MyFun")
Block.declare("Loop", Z, Z, Z)
Block.declare("IfHead", Z, Z, Z)
Block.declare("Then", Z, Z, Z)
Block.declare("Else", Z, Z, Z)
Block.declare("Done", Z,Z,Z)
Block.declare("Ret", Z)
Block = Block.create()

blk = smt.Const("blk", Block)
i,j,k = smt.Consts("i j k", Z)
step = kd.define("step", [blk],
    blk.match_(
        (Block.MyFun, Block.Loop(1,1,0)),
        (Block.Loop(i,j,k), smt.If(k < 100, Block.IfHead(i,j,k), Block.Done(i,j,k))),
        (Block.IfHead(i,j,k), smt.If(j < 20, Block.Then(i,j,k), Block.Else(i,j,k))),
        (Block.Then(i,j,k), Block.Loop(i, i, k + 1)),
        (Block.Else(i,j,k), Block.Loop(i, k, k + 2)),
        (Block.Done(i,j,k), Block.Ret(j)),
        (Block.Ret(i), Block.Ret(i)
    )
))

iterate = smt.Function("iterate", Block, Block)
iterate = kd.define("iterate", [blk], smt.If(blk.is_Ret, blk, iterate(step(blk))))


In [ ]:
Mem = smt.ArraySort(Z,Z)
Block2 = kd.Inductive("Block2", auto_fresh=True)
Block2.declare("MyFun")
Block2.declare("Loop", Mem)
Block2.declare("IfHead", Mem)
Block2.declare("Then", Mem)
Block2.declare("Else", Mem)
Block2.declare("Done", Mem)
Block2 = Block2.create()

blk2 = smt.Const("blk2", Block2)
relate = kd.define("relate",[blk, blk2], 
                   smt.Or(
                          smt.And(blk.is_MyFun, blk2.is_MyFun),
                          smt.And(blk.is_Loop, blk2.is_Loop, blk.Loop.arg0 == blk2.Loop.arg0[0], blk.Loop.arg1 == blk2.Loop.arg0[1], blk.Loop.arg2 == blk2.Loop.arg0[2]),

                   )
))

kd.prove(smt.ForAll([blk,blk2], relate(blk,blk2), relate(step(blk), step2(blk2))))


In [17]:
kd.full_simp(iterate(Block.MyFun))

Ret(1)

In [16]:
kd.full_simp(kd.full_simp(step(step(step(Block.MyFun)))))

Then(1, 1, 0)

In [ ]:
from kdrag.all import *
Z = smt.IntSort()

# predeclare all our blocks so that we can recursively call them
myfun = smt.Function("myfun", Z)
loop = smt.Function("loop", Z,Z,Z, Z)
if_head = smt.Function("if_head", Z,Z,Z, Z)
then = smt.Function("then", Z,Z,Z, Z)
else_ = smt.Function("else_", Z,Z,Z, Z)
done = smt.Function("done", Z,Z,Z, Z)
i,j,k = smt.Ints("i j k")

myfun = kd.define("myfun", [], loop(1,1,0))
loop = kd.define("loop", [i,j,k], smt.If(k < 100, if_head(i,j,k), done(i,j,k)))
if_head = kd.define("if_head", [i,j,k], smt.If(j < 20, then(i,j,k), else_(i,j,k)))
then = kd.define("then", [i,j,k], loop(i, i, k + 1))
else_ = kd.define("else_", [i,j,k], loop(i, k, k + 2))
done = kd.define("done", [i,j,k], j)

two optimizations
Constant folding or dropping j as a parameter
Or swtiching over to an smt.ArraySort(Z,Z) carrying style

In [6]:
import kdrag.theories.fun as fun
from kdrag.all import *
# predeclare all our blocks so that we can recursively call them
Z = smt.IntSort()
Mem = smt.ArraySort(Z,Z)
myfun = smt.Function("myfun", Z)
loop = smt.Function("loop", Mem, Mem)
if_head = smt.Function("if_head", Mem, Mem)
then = smt.Function("then", Mem, Mem)
else_ = smt.Function("else_", Mem, Mem)
done = smt.Function("done", Mem, Mem)
mem = smt.Array("mem", Z, Z)
i = mem[0]
j = mem[1]
k = mem[2]

myfun = kd.define("myfun", [], loop(fun.MultiStore(mem, 0, 1,1,0))[1])
loop = kd.define("loop", [mem], smt.If(k < 100, if_head(mem), done(mem)))
if_head = kd.define("if_head", [mem], smt.If(j < 20, then(mem), else_(mem)))
then = kd.define("then", [mem], loop(fun.MultiStore(mem, 0, i, i, k + 1)))
else_ = kd.define("else_", [mem], loop(fun.MultiStore(mem, 0, i, k, k + 2)))
done = kd.define("done", [mem], mem)

kd.full_simp(kd.full_simp(myfun))

1

In [ ]:
smt.Implies(inputs, outputs_eq)

In [ ]:
def blocks(start_addr):
    blocks = {}
    todo = [start_addr]
    while todo:
        addr = todo.pop()
        block = blocks[addr]
        insn = addr
        while True:
            insn = ctx.translate(insn)
            block.append(insn)
            if insn.is_branch():
                if not_constant():
                    # Todo. Do symbolic execution here?
                    # Do a value propagation?
                    # I'm like reimplementing angr... is that ok?
                    raise Exception("Indirect branch. More careful analysis needed.")
                else:







In [ ]:
import lark
from kdrag.all import *
from kdrag.contrib.ir import *
grammar = r"""
start : NL* funcdef NL*
funcdef: "function" GLOBAL NL? "{" NL block+ "}"
block: LABEL "(" [param_list] ")" NL instrs jump NL
instrs : instr*
instr: TEMP "=" BASETY OP operand ("," operand)* NL

?jump: call | ite | ret

ite : "ite" operand "," call "," call
// jmp: "jmp" call
//jnz: ("jnz" | "ite") val "," call "," call
ret: "ret" [operand] // make a call?

call : LABEL "(" [call_param_list] ")"

operand: SIGNED_INT | TEMP | GLOBAL
OP: /[A-Za-z][A-Za-z0-9]+/

param_list: param ("," param)*
param : BASETY TEMP

call_param_list : operand ("," operand)*

GLOBAL: /\$[A-Za-z_.][A-Za-z0-9_.]*/
LABEL: /@[A-Za-z_.][A-Za-z0-9_.]*/
TEMP: /%[A-Za-z_.][A-Za-z0-9_.]*/
BASETY: "w" | "l" | "s" | "d" | "m" | "b" // add m for memory

%import common.WS_INLINE
%import common.NEWLINE
%import common.ESCAPED_STRING
%import common.SIGNED_INT
%ignore WS_INLINE
%ignore /#[^\n]*/
NL: NEWLINE+
"""
PARSER = lark.Lark(grammar, start="start", parser="lalr")

example = r"""
function $foo{ #(m %mem){
@entry(w %x, w %y, m %mem)
    %t =w bvadd %x, %y 
    %t2 =w bvadd %t, 42 
    ret %mem
@loop(m %mem, w %x)
    %t3 =w bvneg %x
    @entry(%t3)
}
"""
PARSER.parse(example)

basety = {
    "b" : smt.BoolSort(),
    "w" : smt.BitVecSort(32),
    "l" : smt.BitVecSort(64),
    "m" : smt.ArraySort(smt.BitVecSort(64), smt.BitVecSort(8)),
}

@lark.v_args(inline=True)
class FunctionTransformer(lark.Transformer):
    def __init__(self):
        self.labels : dict[str, smt.FuncDeclRef] = {}
        self.temps : dict[str, smt.ExprRef] = {}
        self.funcs : dict[str, smt.FuncDeclRef] = {}
    def get_temp(self, name : str):
        if name not in self.temps:
            raise Exception(f"Unknown temp {name}")
        return self.temps[name]
    def start(self, nl1, *funcs):
        return funcs[:-1] # nl
    def funcdef(self, name, _params, nl1, *blocks):
        #print("funcdef", blocks)
        return Function("entry",  {k : v for k,v in blocks})
    def block(self, label, params, nl1, insns, jmp, nl2):
        #print("block", label, "params", params, "insn", insns, "jm", jmp)
        return (label.value[1:], Block(params, insns + [jmp]))
    def BASETY(self, ty):
        return basety[ty.value]
    def param(self, ty, name):
        #print(name, name[1:])
        self.temps[name[1:]] = smt.Const(name[1:], ty)
        return smt.Const(name[1:], ty)
    def param_list(self, *params):
        return list(params)
    def operand(self, op):
        #print("operand", op)
        if isinstance(op, lark.Token):
            if op.type == "SIGNED_INT":
                return smt.BitVecVal(int(op.value), 32)
            elif op.type == "TEMP":
                #print(self.temps, op.value)
                return self.temps[op.value[1:]]
            elif op.type == "GLOBAL":
                return smt.Const(op.value[1:], smt.BitVecSort(64))
        else:
            raise Exception("Unknown operand type")
    def instrs(self, *instrs):
        return list(instrs)
    def instr(self, dest, ty, op, *operands):
        #print(operands)
        c = smt.Const(dest.value[1:], ty)
        #print(c)
        if op == "bvadd":
            expr = operands[0] + operands[1]
        elif op == "bvneg":
            expr = -operands[0]
        elif op == "bvsub":
            expr = operands[0] - operands[1]
        elif op == "bveq":
            return operands[0] == operands[1]
        else:
            raise Exception(f"Unknown op {op}")
        dname = dest.value[1:]
        if dname in self.temps:
            raise Exception(f"Reassignment to {dname}")
        self.temps[dname] = expr
        return expr
    def call_param_list(self, *params):
        return list(params)
    def call(self, label, params):
        #print("call", label, params, self.temps)
        f = smt.Function(label.value[1:], *[p.sort() for p in params], Bottom)
        return f(*params)
    def ret(self, v):
        return smt.Function("ret", v.sort(), Bottom)(self.temps[v.decl().name()])
    def ite(self, cond, call_true, call_false):
        return smt.If(cond, call_true, call_false)
    def NL(self, token):
        return None
    #def TEMP(self, name):
    #    #print(name)
    #    #return self.temps[name]


def parse(s : str):
    tree = PARSER.parse(s)
    builder = FunctionTransformer()
    return builder.transform(tree)


parse(example)[0]

sumn = r"""
function $sumn {
@entry(w %n)
    @loop(%n, 0)
@loop (w %i, w %acc)
    %acc1 =w bvadd %acc, %i
    %i1   =w bvsub %i, 1
    %c =b  bveq %i1, 0
    ite %c, @loop(%i1, %acc1), @done(%acc1)
@done(w %res)
    ret %res
}
"""
parse(sumn)[0].blocks["loop"]


VisitError: Error trying to process rule "operand":

'c'

In [ ]:
from dataclasses import dataclass
import pypcode

@dataclass
class Correspondance():
    highlabel : str
    lowaddr : int
    read_vars : list[str, pypcode.Varnode]
    write_vars : list[str, pypcode.Varnode]


In [ ]:
def render_qbe(b : Block):
    # Convert to Phi nodes
    phis =[[] in range(b.arity)]
    for pred in preds(b):
        jmp = pred.insns[-1]
        if smt.is_ite(jmp):
        else:
            if jmp.decl() == self:
                for xs, c in zip(phis, jmp.children)
                    xs.append(c)
        



    for insn in b.insns:
        


In [ ]:
import operator
ops = {
    "bvadd", operator.add,
    "add" : operator.add,
    "sub" : operator.sub,
    ""
}

In [24]:
%%file /tmp/swap.s

.global swap
swap:
    mov %rdi, %rax
    mov %rsi, %rdi
    mov %rax, %rsi
    ret

Writing /tmp/swap.s


In [25]:
!gcc -c -o /tmp/swap /tmp/swap.s

In [ ]:
import angr

proj = angr.Project("/tmp/swap")
proj.loader.find_symbol("swap").rebased_addr
cfg = proj.analyses.CFG()
list(cfg.functions["swap"].blocks)
dict(proj.kb.functions)


WARNING  | 2026-01-28 11:59:16,933 | angr.analyses.cfg.cfg_base | "auto_load_libs" is enabled. With libraries loaded in project, CFG will cover libraries, which may take significantly more time than expected. You may reload the binary with "auto_load_libs" disabled, or specify "regions" to limit the scope of CFG recovery.


{4194304: <Function swap (0x400000)>}

In [ ]:
import kdrag.contrib.pcode as pcode
import kdrag.contrib.pcode.asmspec as asmspec

#help(pcode.BinaryContext)
ctx = pcode.BinaryContext("/tmp/swap")
func = ctx.function("swap")
ctx.block_of_addr("swap")

class Block():
    start_addr : int
    end_addr : int
    insns : list[pcode.Instruction]
    args : list[smt.ArrayRef] # mem, reg, unique
    rets : list[smt.ArrayRef] # mem, reg, unique
    jmps : list[tuple[smt.BoolRef, int]]


Unexpected SP conflict


AttributeError: 'BinaryContext' object has no attribute 'get_function_by_name'

In [7]:
import kdrag.contrib.ir as ir
from kdrag.all import *
help(ir.Block.of_expr)

Help on method of_expr in module kdrag.contrib.ir:

of_expr(e: z3.z3.ExprRef) -> 'Block' class method of kdrag.contrib.ir.Block
    >>> x,y = smt.BitVecs("x y", 64)
    >>> x,y = smt.Var(1, smt.BitVecSort(64)), smt.Var(0, smt.BitVecSort(64))
    >>> z = smt.BitVec("z", 64)
    >>> Block.of_expr(smt.If(True, (x + y)*42, x - y + z))
    ^(bv64,bv64):
    %0 = bvadd %var1, %var0
    %1 = bvmul %0, 42
    %2 = bvsub %var1, %var0
    %3 = bvadd %2, z
    %4 = if True, %1, %3



In [ ]:
from dataclasses import dataclass, field
from kdrag.all import *
class Block():
    name : str
    args : list[smt.ExprRef]
    insns : list[smt.ExprRef]
    jmp : smt.ExprRef # either if x, block1, block2 or just goto block

class Function():
    blocks : list[Block] # 0th block is entry. Or "entry" is entry? Naw. 0th.

Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)


In [ ]:
x = smt.BitVec("x", 64)
(x+x).decl().name()


'bvadd'

In [8]:
smt.is_bv_value(x)

False

In [ ]:
from typing import Union
@dataclass
class LetBlock():
    defns : dict[smt.ExprRef, smt.ExprRef]
    body : Union[smt.ExprRef,"LetBlock"]
    def expand(self):
        if isinstance(self.body, LetBlock):
            body = self.body.expand()
            # Or maybe we should expand into the inner lets, taking care of capture avoidance?
        else:
            body = self.body
        return smt.substitute(body, *self.defns.items())

x,y = smt.Ints("x y")
LetBlock({x : smt.IntVal(3)}, x + 7).expand()


3 + 7

In [75]:
from dataclasses import dataclass, field
import pprint
Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)
mem = smt.Array("mem", smt.BitVecSort(64), smt.BitVecSort(64))
@dataclass
class Block():
    args : list[smt.ExprRef]
    insns : list[smt.ExprRef] = field(default_factory=list)
    @classmethod
    def of_expr(cls, vs, e):
        insns = []
        seen = set(vs)
        todo = [e]
        while todo:
            e = todo.pop()
            insns.append(e)
            for arg in e.children():
                if arg not in seen:
                    seen.add(arg)
                    todo.append(arg)
        return cls(args=vs, insns=list(reversed(insns)))
    def __repr__(self):
        res = ["blk(" + ", ".join("%" + str(a.get_id()) for a in self.args) + "):"]
        for insn in self.insns:
            res.append(f"let %{insn.get_id()} := {insn.decl().name()} " + ", ".join("%" + str(c.get_id()) for c in insn.children()))
        #res.append(f"ret %{self.insns[-1].get_id()}")
        return "\n".join(res)
    def cstr(self):
        res = []
        for insn in self.insns:
            decl = insn.decl().name()
            children = [f"v{c.get_id()}" for c in insn.children()]
            if decl == "ret":
                res.append(f"return {children[0]};")
            elif decl == "bvadd":
                res.append(f"uint64_t v{insn.get_id()} = {children[0]} + {children[1]};")
            elif decl == "bvmul":
                res.append(f"uint64_t v{insn.get_id()} = {children[0]} * {children[1]};")
            elif decl == "ite":
                raise NotImplementedError("ite not implemented yet")
            else:
                res.append(f"uint64_t v{insn.get_id()} = {insn.decl().name()}(" + ", ".join(children) + ");")
        return "\n".join(res)
    def riscv(self):
        res = []
        pass

x = smt.BitVec("x", 64)
#x = smt.BitVecVal(42, 64)
b = Block.of_expr([x], ret(x + x + x*x))

@dataclass
class Function():
    blocks : dict[str, Block] = field(default_factory=dict)
    defns : dict[str, smt.FuncDeclRef] = field(default_factory=dict)
    def add_block(self, name : str, vs, body) -> smt.FuncDeclRef:
        assert name not in self.blocks
        f = kd.define(name, vs, body)
        self.defns[name] = f
        self.blocks[name] = Block.of_expr(vs, body)
        return f
    def cstr(self):
        res = []
        for name, blk in self.blocks.items():
            res.append(f"void {name}(" + ", ".join(f"uint64_t v{a.get_id()}" for a in blk.args) + ") {")
            res.append(blk.cstr())
            res.append("}")
        return "\n\n".join(res)
    def simp(self, *args):
        return kd.full_simp(kd.full_simp(self.defns["entry"](*args)))

m = Function()
last = m.add_block("last", [x], ret(x + x + x*x))
entry = m.add_block("entry", [x], last(x + x))


#print(b.cstr())
print(m.cstr())


sumn = Function()
acc, n = smt.BitVecs("acc n", 64)
done = sumn.add_block("done", [acc], ret(acc))
loop = smt.Function("loop", smt.BitVecSort(64), smt.BitVecSort(64), Bottom)
loop = sumn.add_block("loop", [acc, n], 
            smt.If(n > 0, loop(acc + n, n - 1), done(acc))
            )
entry = sumn.add_block("entry", [n], loop(smt.BitVecVal(0, 64), n)) 
pprint.pprint(sumn)

sumn.simp(10)



void last(uint64_t v345) {

uint64_t v497 = v345 + v345;
uint64_t v467 = v345 * v345;
uint64_t v474 = v497 + v467;
return v474;

}

void entry(uint64_t v345) {

uint64_t v497 = v345 + v345;
uint64_t v444 = last(v497);

}
Function(blocks={'done': blk(%488):
let %507 := ret %488,
                 'entry': blk(%468):
let %469 := bv 
let %462 := loop %469, %468,
                 'loop': blk(%488, %468):
let %469 := bv 
let %513 := bvsgt %468, %469
let %516 := bvadd %488, %468
let %508 := bv 
let %509 := bvsub %468, %508
let %495 := loop %516, %509
let %503 := done %488
let %501 := if %513, %495, %503},
         defns={'done': done, 'entry': entry, 'loop': loop})


ret(55)

In [ ]:
type Loc = str
type Insn = str
def compile_riscv(blk : Block, locs : dict[smt.ExprRef, Loc]):
    res : list[Insn] = []
    for insn in reversed(blk.insns):
        if smt.is_bv_value(insn):
            loc = alloc(x, locs)
            res.append(f"li {loc} {insn.as_long()}")
        else:
            decl = insn.decl().name()
            children = insn.children()
            [alloc(c, locs) for c in children]
            match decl:
                case "bvadd":
                    res.append(f"add {} {}")
                case "bvsub":
                    res.append(f"sub {} {}")
                case "bvmul":
                    res.append(f"mul {} {}")
        if insn in locs:
            del insn
        if is_add
    res.reverse()
Bottom = smt.DeclareSort("Bottom")
ret = smt.Function("ret", smt.BitVecSort(64), Bottom)

[
    x := smt.BitVecVal(42),
    y := x + x,
    z := y * y,
    ret(z)
]

SyntaxError: f-string: valid expression required before '}' (2845523921.py, line 13)

The similarly of a malloc implementation and this alloc are interesting.
We need to alloc going backwards when something is used, and free it when it is defined.


In [ ]:
@dataclass
class Alloc():
    regs : set[str]
    stack : int = 0
    def alloc(self): # take program as argument? Take size as argument?
        if self.regs:
            return self.regs.pop()
        else:
            self.stack += 8
            return f"sp+{self.stack}"
    def free(self, ref):
        if ref.startswith("sp+"):
            # don't free stack space
            # happens at the end
            pass
        else:
            self.regs.add(ref)

def riscv64():
    return Alloc(set(["t0","t1","t2","t3","t4","t5","t6"]))

class CallConv():
    args : list[str]

{"ret" : CallConv(["r1"])}


class SymAlloc():
    live : set(smt.ExprRef)
    stack : smt.ArithRef = smt.IntVal(0)
    constrs : list[smt.BoolRef]
    def alloc(self, sort) -> smt.ExprRef:
        v = 
        self.live.add(smt.)
        self.constraints(smt.Distinct(*self.live))
    def free(self, v : smt.ExprRef):
        self.live.remove(v)

In [ ]:

def alloc(x, locs):
    if x in locs:
        return locs[x]
    else:
        for reg in [f"r{n}" for n in range(32)]:
            if reg not in locs.values():
                return reg
        for i in range(1000):
            raise Exception("ran out of registers")
    

def compile_riscv(blk : Block, locs):
    res = []
    for insn in reversed(blk.insns):
        if smt.is_bv_value(insn):
            loc = alloc(x, locs)
            res.append(f"li {loc} {insn.as_long()}")
        else:
            decl = insn.decl().name()
            children = insn.children()
            [alloc(c, locs) for c in children]
            match decl:
                case "bvadd":
                    res.append(f"add {} {}")
                case "bvsub":
                    res.append(f"sub {} {}")
                case "bvmul":
                    res.append(f"mul {} {}")
                case "store":
                    res.append(f"sw {} {}({})")
                case "select":
                    res.append(f"lw {} {}({})")
        if insn in locs:
            del insn
        if is_add
    res.reverse()
    return res

def riscv_compile(f : Function):
    for b in reversed(blocks):
        


In [ ]:
def compile_riscv(insns : ):
    for insn in reversed(insns):


In [ ]:

@dataclass 
class Call(): # App
    f : str
    args : tuple[Expr, ...]

@dataclass
class Defn():
    name : str
    nargs : int
    body : Expr